In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
import os
family_group = "10.Lycaenidae"
annotated_genome_location = f"/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/{family_group}/1.Blast_result"
species_list = os.listdir(annotated_genome_location)
if "desktop.ini" in species_list:
    species_list.remove("desktop.ini")
# species_list = ["Boloria_euphrosyne","Boloria_selene","Brenthis_daphne","Brenthis_hecate","Brenthis_ino","Dryadula_phaetusa","Dryas_iulia_moderata","Eueides_isabella","Fabriciana_adippe","Heliconius_charithonia","Heliconius_nattereri","Heliconius_sara","Philaethria_dido","Hestina_assimilis"]
# species_list = ["Limenitis_camilla"]
# species_list = ["Argynnis_bischoffii_washingtonia","Boloria_euphrosyne","Boloria_selene","Brenthis_daphne","Brenthis_hecate","Brenthis_ino","Dryadula_phaetusa","Dryas_iulia_moderata","Eueides_isabella","Fabriciana_adippe","Heliconius_charithonia","Heliconius_nattereri","Heliconius_sara","Philaethria_dido","Hestina_assimilis"]
# species_list = ["Bicyclus_anynana","Coenonympha_glycerion","Elymnias_hypermnestra","Erebia_aethiops","Erebia_ligea","Hipparchia_semele","Lasiommata_megera","Maniola_hyperantus","Maniola_jurtina","Melanargia_galathea","Oeneis_ivallda","Pararge_aegeria"]
# species_list = ["Erebia_ligea","Hipparchia_semele","Lasiommata_megera","Maniola_hyperantus","Maniola_jurtina","Melanargia_galathea","Oeneis_ivallda","Pararge_aegeria"]
# species_list = ["Vanessa_tameamea"]
for species in species_list:
    print(species)
    blast_location = f"/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/{family_group}/1.Blast_result"
    
    list_of_files = os.listdir(f"{blast_location}/{species}")
    query_species_list = []
    coordinate_detail_dictionary = {}
    coordinate_info_dictionary = {}
    scaffold = ''
    for file in list_of_files:
        if file.endswith(".csv") and len(file.split("_")) >= 5 and not(file.endswith("_old.csv")):
#             print(file)
            if "." in file[:-4]:
                query_species = f"{file.split('_')[-2].split('.')[1]}_{file.split('_')[-1].split('.')[0]}"
            else:
                query_species = f"{file.split('_')[-2]}_{file.split('_')[-1].split('.')[0]}"
            query_species_list.append(query_species)
            coordinate_detail_dictionary[query_species] = {}
            with open(f"{blast_location}/{species}/{file}", 'r') as coordinate_file_open:
                coordinate_file_lines_list  =  coordinate_file_open.readlines()
            if scaffold == '':
                scaffold = coordinate_file_lines_list[1].split(",")[1]
    
            for i in range(1,len(coordinate_file_lines_list)):
                
                coordinate_file_lines_split = coordinate_file_lines_list[i].split(",")
                start,stop = coordinate_file_lines_split[2],coordinate_file_lines_split[3]
                exon = f"Exon_{i}"
                if exon not in coordinate_info_dictionary:
                    coordinate_info_dictionary[exon] = {}
                coordinate_info_dictionary[exon][query_species] = [start,stop]
                coordinate_detail_dictionary[query_species][exon] = coordinate_file_lines_list[i]
            
            # break
            # print(query_species)
            # print(file)
    # print(coordinate_info_dictionary)
    # assert False
    output = coordinate_file_lines_list[0]
    for exon,species_specific_coordinates in coordinate_info_dictionary.items():
        # print(exon,species_specific_coordinates)
        coordinates_list = []
        previous_coordinates = ""
        match = 1
        for query_species, coordinates in coordinate_info_dictionary[exon].items():
            coordinates_list.append([query_species,coordinates])
            if previous_coordinates != '':
                if coordinates != previous_coordinates:
                    match = 0
                    
            previous_coordinates = coordinates
       
        
        
        if match == 0:
            print(exon,"mismatch")
            print(coordinates_list)
                
            while True:
                choice = input("Proceed?")
                if choice[0].lower() == "y" or choice[0].lower() == "n":
                    break
            
            while True:
                if choice[0].lower() == "y":
                    alignment_detail_dictionary = {}
                    for query_species_name in query_species_list:
                        list_of_folders  = os.listdir(f"{blast_location}/{species}")
                        for folders in list_of_folders:
                            if folders.endswith(query_species_name):
                                # print()
                                
                                with open(f"{blast_location}/{species}/{folders}/{exon}/for_blast/new_query.txt", 'r') as query_file_open:
                                    # alignment_detail = query_file_open.readlines()[0]
                                    alignment_detail = query_file_open.readlines()[0].split("set")[1][:-1]
                                    alignment_detail_dictionary[query_species_name] = alignment_detail
                                with open (f"{blast_location}/{species}/{folders}/{exon}/for_alignment/{exon}_translated_genomic_sequence_{alignment_detail}.fa.hat2", 'r') as alignment_score_file:
                                    distance_score = float(alignment_score_file.readlines()[-1].rstrip())
                                try:
                                    with open (f"{blast_location}/{species}/{folders}/{exon}/for_blast/new_query_spliced.txt", 'r') as new_query_file:
                                        new_query = (new_query_file.readlines()[-1].rstrip())
                                except:
                                    new_query = "Issue with query"
                                print(query_species_name,exon, alignment_detail, distance_score,f"\n{new_query}")
                    while True:            
                        check_alignment = input("Check Alignment??")
                        if check_alignment[0].lower() == "y" or check_alignment[0].lower() == "n":
                            break
                    if check_alignment[0].lower() == "y":
                        for query_species_name in query_species_list:
                            list_of_folders  = os.listdir(f"{blast_location}/{species}")
                            for folders in list_of_folders:
                                if folders.endswith(query_species_name):
                                    with open (f"{blast_location}/{species}/{folders}/{exon}/for_alignment/alignment_clustal_{exon}_translated_genomic_sequence_{alignment_detail_dictionary[query_species_name]}.fa.txt", 'r') as clustal_alignment_file:
                                        clustal_alignment = clustal_alignment_file.readlines()
                                    print("".join(clustal_alignment))
                                    
                                # assert False
                    while True:
                        try:
                            species_number = (input(f"Choose Species 1 - {len(query_species_list)}"))
                            
                            
                            if species_number[0].lower() == 'n':
                                output +=  f'{species},{scaffold},000,000,0,Y,Error_{exon},00,00,00\n'
                                break 
                            output += coordinate_detail_dictionary[query_species_list[int(species_number)-1]][exon]
                            break
                        except:
                            print("Retry")
                    break
                elif choice[0].lower() == "n":
                    output +=  f'{species},{scaffold},000,000,0,Y,Error_{exon},00,00,00\n'
                    break
                
                
        else:
            print(exon, "Match")
            print(coordinates_list)
            output += coordinate_detail_dictionary[query_species_list[0]][exon]
        
       

        # assert False
    # print(coordinate_detail_dictionary)
    print(output)
    with open(f"{blast_location}/{species}/final_coordinates.csv", "w") as out_file:
        out_file.write(output)
    
    # 'Coenonympha_glycerion,OY979626.1,7015875,7015963,0,N,5.Bicyclus_anynana_XM_024088150.2_Frame_0_rightoh_2_query_Exon_1,1,29,29\n'

Aricia_agestis
Exon_1 Match
[['Aricia_agestis', ['33839821', '33839909']], ['Maniola_jurtina', ['33839821', '33839909']], ['Danaus_plexippus', ['33839821', '33839909']]]
Exon_2 Match
[['Aricia_agestis', ['33826718', '33826792']], ['Maniola_jurtina', ['33826718', '33826792']], ['Danaus_plexippus', ['33826718', '33826792']]]
Exon_3 Match
[['Aricia_agestis', ['33826164', '33826336']], ['Maniola_jurtina', ['33826164', '33826336']], ['Danaus_plexippus', ['33826164', '33826336']]]
Exon_4 mismatch
[['Aricia_agestis', ['33824396', '33824493']], ['Maniola_jurtina', ['33825720', '33825895']], ['Danaus_plexippus', ['33824396', '33824493']]]
Proceed?y
Aricia_agestis Exon_4 72_frame0 0.0 
YLSSPPPTVGEGGNELDNMDISTSDKSTRQNK
Maniola_jurtina Exon_4 24_frame2 1.657 
RKVVSLASRIIVLTLIVSLSECNLNGRRPKRKSAHVNPNIEPSAVEATNTKTIIYWKF
Danaus_plexippus Exon_4 72_frame0 1.486 
YLSSPPPTVGEGGNELDNMDISTSDKSTRQNK
Check Alignment??y
CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Aricia_agestis_ -----YLSSPPPTVGEGGNEL

Proceed?y
Aricia_agestis Exon_18 54_frame0 0.0 
PHVPIYYTPLQPQAIPVLENYPPNPFASTILPIPTTQA
Maniola_jurtina Exon_18 54_frame0 1.479 
PHVPIYYTPLQPQAIPVLENYPPNPFASTILPIPTTQA
Danaus_plexippus Exon_18 54_frame0 1.573 
INVFVFSAPHVPIYYTPLQPQAIPVLENYPPNPFASTILPIPTTQA
Check Alignment??y
CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Aricia_agestis_ -----------PHVPIYYTPLQPQAIPVLENYPPNPFASTILPIPTTQA-----------
set54_frame0    AVDINVFVFSAPHVPIYYTPLQPQAIPVLENYPPNPFASTILPIPTTQAGLSTGTLYRSL
                           **************************************           

Aricia_agestis_ ------
set54_frame0    LTYRSF
                      

CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


28.Maniola_jurt A------------LPMYYTPTSTQQMP--SRFNPN----TNLP---------------HA
set54_frame0    AVDINVFVFSAPHVPIYYTPLQPQAIPVLENYPPNPFASTILPIPTTQAGLSTGTLYRSL
                *            :*:**** ..* :*  ..: **    * **                 

28.Maniola_jurt HYHRYQ
set54_frame0    LTYRSF
                  :*  

CLUSTAL format 

Exon_1 Match
[['Aricia_agestis', ['36564445', '36564533']], ['Maniola_jurtina', ['36564445', '36564533']], ['Danaus_plexippus', ['36564445', '36564533']]]
Exon_2 mismatch
[['Aricia_agestis', ['36540477', '36540551']], ['Maniola_jurtina', ['36540477', '36540653']], ['Danaus_plexippus', ['36540477', '36540653']]]
Proceed?y
Aricia_agestis Exon_2 474_frame1 0.0 
SKSTSESNSSGSSGYRGKRTDADY
Maniola_jurtina Exon_2 474_frame1 0.971 
DDTTMLLLFNFFSFLTDYKYSCVKIVISKFSCFSSKSTSESNSSGSSGYRGKRTDADY
Danaus_plexippus Exon_2 474_frame1 0.985 
DDTTMLLLFNFFSFLTDYKYSCVKIVISKFSCFSSKSTSESNSSGSSGYRGKRTDADY
Check Alignment??y
CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Aricia_agestis_ -------------------------------------SKSTSESNSSGSSGYRGKRTDAD
set474_frame1   VKSDDTTMLLLFNFFSFLTDYKYSCVKIVISKFSCFSSKSTSESNSSGSSGYRGKRTDAD
                                                     ***********************

Aricia_agestis_ Y---
set474_frame1   YMYA
                *   

CLUSTAL format alignment by MAFFT L-INS-i (v7

Proceed?y
Aricia_agestis Exon_17 67_frame2 0.0 
LNGLKRPTDSMSCPIPCKTRCTVVNGDRTKSVAMVPVFPNAAI
Maniola_jurtina Exon_17 67_frame2 1.694 
LNGLKRPTDSMSCPIPCKTRCTVVNGDRTKSVAMVPVFPNAAIGK
Danaus_plexippus Exon_17 67_frame2 1.632 
LNGLKRPTDSMSCPIPCKTRCTVVNGDRTKSVAMVPVFPNAAI
Check Alignment??y
CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Aricia_agestis_ --LNGLKRPTDSMSCPIPCKTRCTVVNGDRTKSVAMVPVFPNAAI---
set67_frame2    LQLNGLKRPTDSMSCPIPCKTRCTVVNGDRTKSVAMVPVFPNAAIGKC
                  *******************************************   

CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


28.Maniola_jurt --AHGLKRASKQTEDGTTHKHRCSSPRNRRRFSQKVTNSPPLQATNN-
set67_frame2    LQLNGLKRPTDSMSCPIPCKTRCTVVNGDRTKSVAMVPVFPNAAIGKC
                   :****.:.. .   . * **:  .. *  *  :.   *  * .: 

CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


13.Danaus_plexi KLPIQPHGVKRPSKQAEEASAHKHRCSSPRPIRHSAVSNNQPV----AINSV
set67_frame2    ---LQLNGLKRPTDSMSCPIPCKTRCTVVNGDRTKSVA-MVPVFPNAAIGKC
                   :* :*:**

Proceed?y
Aricia_agestis Exon_4 231_frame2 0.976 
YLTPPPASVGDVGKETEIMDVSTSENKARLNN
Maniola_jurtina Exon_4 88_frame1 1.571 
SF
Danaus_plexippus Exon_4 71_frame2 1.597 
IVSHNLQVLWLSR
Check Alignment??y
CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Aricia_agestis_ -----YLSSPPPTVGEGGNELDNMDISTSDKSTRQNK----
set231_frame2   CFSAAYLTPPPASVGDVGKETEIMDVSTSENKARLNNVSIY
                     **:.**.:**: *:* : **:***::.:* *:    

CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


28.Maniola_jurt AQTQSPGLVQPEKGPETMEVSDRTSGKDE---------------
set88_frame1    -------LFKPDLHYETSFMHDKTSFMKQIVAHVKLLSLKIIVS
                       *.:*:   **  : *:**  .:               

CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


13.Danaus_plexi NR--------TPTPSLVQTEKGPENMEITTLKR--------NDEKDE
set71_frame2    SAPRWRCPLPPPRPLLYVTTIVSHNLQVLWLSRVGHCPVVMTTHLYY
                .         .* * *  *   ..*:::  *.*        . .   

Choose Species 1 - 31
Exon_5 Match
[['Aricia_agestis', ['151271', '151375']], ['Maniola_j

...

Choose Species 1 - 31
Exon_16 mismatch
[['Aricia_agestis', ['6757168', '6757342']], ['Maniola_jurtina', ['6757168', '6757375']], ['Danaus_plexippus', ['6757168', '6757375']]]
Proceed?y
Aricia_agestis Exon_16 22_frame2 0.142 
HNEKMEKKLVKTHYKNKSLIKGEKGKSKEIKITKQKMNEHLDRCNKSYQPTAGDHLAH
Maniola_jurtina Exon_16 22_frame2 1.138 
HNEKMEKKLVKTHYKNKSLIKGEKGKSKEIKITKQKMNEHLDRCNKSYQPTAGDHLAHVSPPLYYYYHT
Danaus_plexippus Exon_16 22_frame2 1.211 
HNEKMEKKLVKTHYKNKSLIKGEKGKSKEIKITKQKMNEHLDRCNKSYQPTAGDHLAHVSPPLYYYYHT
Check Alignment??y
CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Aricia_agestis_ ------HNEKMEKKLVKTHYKNKSLMKGERGKSKEIKITKQKNNEILDRCNKSYQPTAGD
set22_frame2    YNNNSRHNEKMEKKLVKTHYKNKSLIKGEKGKSKEIKITKQKMNEHLDRCNKSYQPTAGD
                      *******************:***:************ ** **************

Aricia_agestis_ HLAH-------------
set22_frame2    HLAHVSPPLYYYYHTVL
                ****             

CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


28.Maniola_jurt ------HNIDMEKELLK

Exon_1 Match
[['Aricia_agestis', ['30203934', '30204022']], ['Maniola_jurtina', ['30203934', '30204022']], ['Danaus_plexippus', ['30203934', '30204022']]]
Exon_2 Match
[['Aricia_agestis', ['30196526', '30196600']], ['Maniola_jurtina', ['30196526', '30196600']], ['Danaus_plexippus', ['30196526', '30196600']]]
Exon_3 mismatch
[['Aricia_agestis', ['30195464', '30195636']], ['Maniola_jurtina', ['30203278', '30203339']], ['Danaus_plexippus', ['30195464', '30195636']]]
Proceed?y
Aricia_agestis Exon_3 179_frame2 0.474 
KTNNAQPIIEKKIINDETTAEVKLKSVEVVTNHDAQTIESKDTSPSIVEEVKDGSDN
Maniola_jurtina Exon_3 12_frame0 1.74 
TREREKCVARFFITADQKEG
Danaus_plexippus Exon_3 179_frame2 1.72 
KTNNAQPIIEKKIINDETTAEVKLKSVEVVTNHDAQTIESKDTSPSIVEEVKDGSDN
Check Alignment??y
CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Aricia_agestis_ --------KNNNAQPIIEKRVNDEIKTAEVKLKSAEVVNNHDAQTIESKDTSPPTTEDVK
set179_frame2   FFRTYNYRKTNNAQPIIEKKIINDETTAEVKLKSVEVVTNHDAQTIESKDTSPSIVEEVK
                        *.*********:: :

Choose Species 1 - 31
Exon_15 Match
[['Aricia_agestis', ['30185981', '30186112']], ['Maniola_jurtina', ['30185981', '30186112']], ['Danaus_plexippus', ['30185981', '30186112']]]
Exon_16 Match
[['Aricia_agestis', ['30185249', '30185423']], ['Maniola_jurtina', ['30185249', '30185423']], ['Danaus_plexippus', ['30185249', '30185423']]]
Exon_17 mismatch
[['Aricia_agestis', ['30183860', '30183989']], ['Maniola_jurtina', ['30184396', '30184480']], ['Danaus_plexippus', ['30183860', '30183989']]]
Proceed?y
Aricia_agestis Exon_17 45_frame2 0.057 
LNGLKRPTDSMSCPIPCKTRCTVVNGDRTKSVGMMPVFPNAAI
Maniola_jurtina Exon_17 41_frame0 1.642 
IKPNFEHDANSCCESPRSRLRVKENGKS
Danaus_plexippus Exon_17 45_frame2 1.635 
LNGLKRPTDSMSCPIPCKTRCTVVNGDRTKSVGMMPVFPNAAI
Check Alignment??y
CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Aricia_agestis_ --LNGLKRPTDSMSCPIPCKTRCTVVNGDRTKSVAMVPVFPNAAI---
set45_frame2    LQLNGLKRPTDSMSCPIPCKTRCTVVNGDRTKSVGMMPVFPNAAIGKC
                  ********************************.*:**

Danaus_plexippus Exon_3 198_frame2 1.657 
RIQQECTAARRHYKRYRRRRTRDQQEEEDLYTAYRTACLTLRNAILQAKADAWQDWLRTLEEDPWGRPYKWVRQKLRPAAAPITQTIEPEIRGRVLSTLFPRGANCSPLVAPR
Check Alignment??y
CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Aricia_agestis_ --------KNNNAQPIIEKRVNDEIKTAEVKLKSAEVVNNHDAQTIESKDTSPPTTEDVK
set336_frame2   FFPTYNYRKTNNAQPIIEKRVNDD-TTAEVKLKSVEVVTNHDAQTIESNDTSPSIVEEVK
                        *.*************: .********.***.*********:****. .*:**

Aricia_agestis_ DGSDN---------
set336_frame2   DGSDNGETLKLLKF
                *****         

CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


28.Maniola_jurt I--------NNESQPPEKRSKDKDAKKKIPLQGLQTDLKV-TEVINENECVPEIAPIFES
set336_frame2   FFPTYNYRKTNNAQPIIEKRVNDDTTAEVKLKSVEVVTNHDAQTIESNDTSPSIVEEVKD
                :        .*::**  ::  :.*:. :: *:.::.  :  ::.*:.*:  *.*.  .:.

28.Maniola_jurt SKED------ISD
set336_frame2   GSDNGETLKLLKF
                ..::      :. 

CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


13.Danaus_plexi N

Choose Species 1 - 31
Exon_15 mismatch
[['Aricia_agestis', ['31373552', '31373683']], ['Maniola_jurtina', ['31373552', '31373683']], ['Danaus_plexippus', ['31373552', '31373668']]]
Proceed?y
Aricia_agestis Exon_15 11_frame2 0.187 
VAPTRNAKVSPMPSEQTTTSTNSSPNSTILQNKTVWLTESLLN
Maniola_jurtina Exon_15 1_frame0 1.589 
Issue with query
Danaus_plexippus Exon_15 11_frame2 1.587 
NAKVSPMPSEQTTTSTNSSPNSTILQNKTVWLTESLLN
Check Alignment??y
CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Aricia_agestis_ ---------------------------------------VPPTTNAKVSPIPSEQTTTST
set11_frame2    YLLTILDANNCIYITKLGDQKKFFFVYHFTIVKQICYFRVAPTRNAKVSPMPSEQTTTST
                                                       *.** ******:*********

Aricia_agestis_ SSSPNSTVLQNKTVWLTESLLN-
set11_frame2    NSSPNSTILQNKTVWLTESLLNK
                .******:************** 

CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


28.Maniola_jurt RC-------ILSNPGELDERASTSDSNAAAGASTSRAGTSIEYPALRLTESLLN
set1_frame0     ETESIKLGGILSHCTE------